In [341]:
import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup
from openpyxl import Workbook, load_workbook

from itertools import product

In [342]:
stroke_count_html = requests.get('https://en.wikipedia.org/wiki/Radical_214')
# stroke_count_html

In [343]:
stroke_count_bs = BeautifulSoup(stroke_count_html.text)
# stroke_count_bs

In [344]:
stroke_count_table = stroke_count_bs.find_all(name = 'table')[5]
# stroke_count_table

In [345]:
stroke_count_table_tbody = stroke_count_table.find_all(name = 'tbody')
# stroke_count_table_tbody

In [346]:
stroke_count_table_tr = stroke_count_table_tbody[0].find_all(name = 'tr')
stroke_count_table_tr[17].text # get 17 strokes column 

'17 strokes\n214 龠\n'

In [347]:
print(stroke_count_table_tr[1].text.split('\n'))

['1 stroke', '1 一', '2 丨', '3 丶', '4 丿', '5 乙', '6 亅', '']


In [348]:
meaning_html = requests.get('http://hanzidb.org/radicals')

In [349]:
meaning_bs = BeautifulSoup(meaning_html.text)
# meaning_bs

In [350]:
meaning_table = meaning_bs.find_all(name = 'table')[0]
# meaning_table

In [351]:
meaning_table_tr = meaning_table.find_all(name = 'tr')
# meaning_table_tr
# meaning_table_tr[1].text

In [352]:
meaning_table_th = meaning_table_tr[1].find_all(name = 'td')
# meaning_table_th

In [353]:
print(meaning_table_tr[10].find_all(name = 'td')[4].text)

son, legs


In [354]:
indices = dict()
for i in range(1, 18):
    row_list = stroke_count_table_tr[i].text.split('\n')
    indices[i] = []
    
    for j in range(1, len(row_list) - 1):
        num, rad = row_list[j].split()
        num = int(num)
        defi = meaning_table_tr[num].find_all(name = 'td')[4].text
        
        item = {"radical": rad, "definition": defi}
        indices[i].append(item)


# indices

In [355]:
wb2 = load_workbook('pre-design.xlsx')

radical = wb2['radical']
kanji = wb2['kanji']

In [356]:
# Strokes count link
# print(chr(ord(init_col) + 1))
# for i in range(9):

# for STROKE 1 ALONE
radical['B8'].hyperlink = "#{name}!A{row}".format(name = radical.title, row = 10)

init_col = 'C'
init_row = 55

for i in range(8):
    cell_str = init_col + '8'
    
    radical[cell_str].hyperlink = "#{name}!A{row}".format(name = radical.title, row = init_row)
    print("#{name}!A{row}".format(name = radical.title, row = init_row))
    
    init_col = chr(ord(init_col) + 1)
    init_row = init_row + 24
    

#radical!A55
#radical!A79
#radical!A103
#radical!A127
#radical!A151
#radical!A175
#radical!A199
#radical!A223


In [357]:
init_col = 'A'
init_row = 10

for i in range(9):
    cell_str = 'A' + str(init_row)
    print(cell_str)
    
    radical[cell_str].hyperlink = "#{name}!A{row}".format(name = 'radical', row = 1)  
    init_row = init_row + 24


A10
A34
A58
A82
A106
A130
A154
A178
A202


In [358]:
s1data = {}

row = 12
for k in range(1, 9):
    ctr = 0
    for i in range(5):
        start = 65
        for j in range(11):
            alp = chr(start + j)
            cell = alp + str(row)
            s1data[alp + str(row)] = {'radical': '', 'definition': ''}
            
            if(ctr < len(indices[k])):
                s1data[alp + str(row)] = indices[k][ctr]
                ctr += 1
            
        row += 4
    row += 4

In [359]:
row

204

In [360]:
row = 204
more_strokes = []

for k in range(9, 18):
    more_strokes.extend(indices[k])
    
ctr = 0
for i in range(5):
    start = 65
    for j in range(11):
        alp = chr(start + j)
        cell = alp + str(row)
        s1data[alp + str(row)] = {'radical': '', 'definition': ''}

        if(ctr < len(more_strokes)):
            s1data[alp + str(row)] = more_strokes[ctr]
            ctr += 1

    row += 4    


In [361]:
columns = [chr(i) for i in range(65, 91)]

keywords = product(columns, repeat = 2)
keywords = [''.join(i) for i in keywords]

columns.extend(keywords)

# columns

In [362]:
ctr = 1 # starts from B babe!
for i in s1data.keys():
    meaning_str = i[0] + str(int(i[1:]) + 3)
    
    radical[i] = s1data[i]['radical']
    radical[meaning_str] = s1data[i]['definition']
    
    if(s1data[i]['radical'] != ''):
        radical[i].hyperlink = "#{name}!{col}8".format(name = 'kanji', col = columns[ctr])
        kanji[columns[ctr] + '8'] = s1data[i]['radical']
        ctr += 1

In [363]:
wb2.save('kodo-senshi-kanji-radicals-cluster-ms-excel.xlsx')